In [1]:
from sklearn.model_selection import train_test_split
from data_pipeline import Pipeline
from metrics import Metrics 
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dunca\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dunca\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('amazon_movie_reviews.csv') 

C:\Users\dunca\AppData\Local\Temp\ipykernel_30156\1828681962.py:1: DtypeWarning: Columns (18,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('amazon_movie_reviews.csv')


In [3]:
n_samples = 10000 #Dataset is inbalanced so this is a good way to ensure proper balancing of ratings
min_count = df['rating'].value_counts().min()
n_samples = min(n_samples, min_count) 
print(n_samples)

df = df.groupby('rating').apply(lambda x: x.sample(n_samples)).reset_index(drop=True)

10000


C:\Users\dunca\AppData\Local\Temp\ipykernel_30156\913442077.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('rating').apply(lambda x: x.sample(n_samples)).reset_index(drop=True)


In [4]:
pipeline = Pipeline()
transformed_data = pipeline.transform(df)

In [5]:
X = transformed_data.drop(['rating', 'polarity'], axis=1)  # Drop the rating and polarity to avoid leakage
y = transformed_data['polarity']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
metrics = Metrics()

In [11]:
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [12]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100)

model.fit(X_train, y_train)  # Train the model
y_pred = model.predict(X_test)  # Predict on the test data

: 

In [ ]:
metric = metrics.calculate_metrics(y_test, y_pred)  
print(f"Metrics for Model:")
for m, value in metric.items():
    print(f"{m}: {value:.4f}")

Metrics for Model:
MAE: 0.1445
MSE: 0.0457
RMSE: 0.2137
R^2: 0.6263


In [12]:
from joblib import dump

dump(model, 'random_forest_model_not24.joblib') 
dump(pipeline.vectorizer, 'tfidf_vectorizer.joblib')

/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


['umap_model.joblib']